In [1]:
from mpasanalysis import *
from e3sm_simulations import *

In [2]:
# get paths of restart files, monthly mean output files, processed climatology files and output figures
ts_yr0 = 26
ts_yr1 = 55
plt_yr0 = 26
plt_yr1 = 55
rest_yr = 51
nmon = 12 # 12 for production and 1 for testing
run = E3SMSimulation(database='e3sm_simulations.xml', runname='low-res-cmp')
data_root = run.get_path(comp='ocn', climo_yr0=ts_yr0, climo_yr1=ts_yr1,
                         ts_yr0=ts_yr0, ts_yr1=ts_yr1, rest_yr=rest_yr)
rst_root = data_root['rst_root']
mon_root = data_root['mon_root']
fig_root = data_root['fig_root']
rst_file = rst_root+'/mpaso.rst.{:04d}-01-01_00000.nc'.format(rest_yr)

# load mesh
mpasmesh = MPASMesh(filepath=rst_file)

# transects
transect_list = ['AR7W',
                 'Davis Strait',
                 'Hudson Strait',
                 'Nares Strait',
                 'Parry Channel',
                 'Jones Sound']

# transect
for transect_name in transect_list:
    trnsct = VerticalTransect(transect_name)
    path0 = mpasmesh.get_shortest_path(trnsct.lon0, trnsct.lat0, trnsct.lon1, trnsct.lat1, npoint_ref=2)

    nv = len(path0.idx_vertex)
    transport = np.zeros([plt_yr1-plt_yr0+1, nmon, nv])
    for i, y in enumerate(np.arange(plt_yr1-plt_yr0+1)+plt_yr0):
        print('{:04d}'.format(y))
        for j, m in enumerate(np.arange(nmon)+1):
#             print('{:04d}-{:02d}'.format(y, m))
            mon_file = mon_root+'/mpaso.hist.am.timeSeriesStatsMonthly.{:04d}-{:02d}-01.nc'.format(y, m)
#             print(mon_file)
            mpasdata = MPASOData(filepath=mon_file, filepath_mesh=rst_file)
            transport0, dist0 = mpasdata.get_transport_cumulative(path=path0, varname_prefix='timeMonthly_avg_',
                                                                  bolus=True)
            transport[i,j,:] = transport0[0]
    # dist = dist0
    dist = gc_distance(np.array(path0.lon_vertex), np.array(path0.lat_vertex), trnsct.lon0, trnsct.lat0)

    # save data
    path_transport = fig_root+'/data_transport_LabSea_'+transect_name.replace(" ", "-")+'_volume.npz'
    np.savez(path_transport, transport=transport, distance=dist)
    

Pre-defined transect 'AR7W'.
Vertex closest to P0: 304.10352 53.30078
Vertex closest to P1: 312.08908 60.99671
 - Vertex closest to RefP1: 306.25754 55.97118
 - Vertex closest to RefP2: 308.91041 58.65620
0026
0027
0028
0029
0030
0031
0032
0033
0034
0035
0036
0037
0038
0039
0040
0041
0042
0043
0044
0045
0046
0047
0048
0049
0050
0051
0052
0053
0054
0055
Pre-defined transect 'Davis Strait'.
Vertex closest to P0: 298.25620 66.64992
Vertex closest to P1: 305.97253 67.01286
 - Vertex closest to RefP1: 300.84487 66.86094
 - Vertex closest to RefP2: 303.35423 66.82543
0026
0027
0028
0029
0030
0031
0032
0033
0034
0035
0036
0037
0038
0039
0040
0041
0042
0043
0044
0045
0046
0047
0048
0049
0050
0051
0052
0053
0054
0055
Pre-defined transect 'Hudson Strait'.
Vertex closest to P0: 295.16686 60.31463
Vertex closest to P1: 293.86939 62.00006
 - Vertex closest to RefP1: 294.62793 61.02786
 - Vertex closest to RefP2: 294.44917 61.57737
0026
0027
0028
0029
0030
0031
0032
0033
0034
0035
0036
0037
0038
003